In [1]:
#importing the important libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
#reading the data
df = pd.read_csv("model_data.csv", sep=";")
df.head()

,product_tier,make_name,price,first_zip_digit,first_registration_year,search_views,detail_views,stock_days,ctr,created_month,deleted_month
0,Basic,Mitsubishi,16750,5,2013,3091,123,31,0.037803,7,8
1,Basic,Mercedes-Benz,35950,4,2015,3283,223,52,0.067926,8,10
2,Basic,Mercedes-Benz,11950,3,1998,3247,265,51,0.081614,7,9
3,Basic,Ford,1750,6,2003,1856,26,101,0.014009,7,10
4,Basic,Mercedes-Benz,26500,3,2014,490,20,11,0.040816,8,9


To get the data ready for modelling we need to:

- Drop the "search_views" column to avoid multi-collinearity.

- Get numerical values for the categorical data (one hot encoding).

- Get a scaled copy of the data.

- Split the dataset into training and testing set and seperate the features from the target value. 

In [3]:
#dropping the search views and the duplicates
df.drop("search_views", axis=1, inplace=True)

df.drop_duplicates(inplace=True)

In [4]:
df.duplicated().sum()

0

In [5]:
#splitting features and target values
X = df.iloc[:, 1:]

y = df.iloc[:, 0]

In [6]:
X.head()

,make_name,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month
0,Mitsubishi,16750,5,2013,123,31,0.037803,7,8
1,Mercedes-Benz,35950,4,2015,223,52,0.067926,8,10
2,Mercedes-Benz,11950,3,1998,265,51,0.081614,7,9
3,Ford,1750,6,2003,26,101,0.014009,7,10
4,Mercedes-Benz,26500,3,2014,20,11,0.040816,8,9


In [7]:
y.head()

0    Basic
1    Basic
2    Basic
3    Basic
4    Basic
Name: product_tier, dtype: object

In [8]:
X.make_name.nunique()

91

- The column "make_name" has so many categories that if we on hot encoded them, we shall increase the dimensionality drastically.

- I will choose only the top 20 classes and one hot encode them, and give 0 for the rest of the classes.

In [9]:
#finding the top 20 classes
top20 = X.make_name.value_counts(ascending=False).head(20).index

In [10]:
#assigning the other classes to the value "n"
X.make_name = X.make_name.apply(lambda x: x.replace(x,"n") if x not in top20 else x)

In [11]:
X.make_name.value_counts(ascending=False)

Volkswagen       9417
n                7132
Renault          6929
Peugeot          5444
Opel             5215
Ford             5196
Mercedes-Benz    4757
BMW              4551
Volvo            3992
Toyota           3924
Audi             3311
Citroen          3017
Fiat             2586
Nissan           2015
Kia              1931
Hyundai          1802
SEAT             1772
Skoda            1515
Suzuki           1312
MINI             1234
Mazda            1198
Name: make_name, dtype: int64

In [12]:
X.head()

,make_name,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month
0,n,16750,5,2013,123,31,0.037803,7,8
1,Mercedes-Benz,35950,4,2015,223,52,0.067926,8,10
2,Mercedes-Benz,11950,3,1998,265,51,0.081614,7,9
3,Ford,1750,6,2003,26,101,0.014009,7,10
4,Mercedes-Benz,26500,3,2014,20,11,0.040816,8,9


In [13]:
#getting dummy variables for the categorical values
X = pd.get_dummies(X, drop_first=True)

In [14]:
X.head()

,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month,make_name_BMW,make_name_Citroen,...,make_name_Opel,make_name_Peugeot,make_name_Renault,make_name_SEAT,make_name_Skoda,make_name_Suzuki,make_name_Toyota,make_name_Volkswagen,make_name_Volvo,make_name_n
0,16750,5,2013,123,31,0.037803,7,8,0,0,...,0,0,0,0,0,0,0,0,0,1
1,35950,4,2015,223,52,0.067926,8,10,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11950,3,1998,265,51,0.081614,7,9,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1750,6,2003,26,101,0.014009,7,10,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26500,3,2014,20,11,0.040816,8,9,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#removing the make_name part from the column names
X.columns = X.columns.str.replace('make_name_', '')

In [16]:
X.columns

Index(['price', 'first_zip_digit', 'first_registration_year', 'detail_views',
       'stock_days', 'ctr', 'created_month', 'deleted_month', 'BMW', 'Citroen',
       'Fiat', 'Ford', 'Hyundai', 'Kia', 'MINI', 'Mazda', 'Mercedes-Benz',
       'Nissan', 'Opel', 'Peugeot', 'Renault', 'SEAT', 'Skoda', 'Suzuki',
       'Toyota', 'Volkswagen', 'Volvo', 'n'],
      dtype='object')

Because of the imbalance in the data, I will perform a stratified splitting to keep the distribution of the target variable in all the splits. Then, I will try oversampling techniques to boost the model's performance.

In [17]:
#Data splitting 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [18]:
X_train

,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month,BMW,Citroen,...,Opel,Peugeot,Renault,SEAT,Skoda,Suzuki,Toyota,Volkswagen,Volvo,n
63518,12990,3,2009,394,117,0.053781,7,11,0,0,...,0,0,0,0,0,0,0,0,0,1
49793,24950,5,1992,151,15,0.040256,11,11,1,0,...,0,0,0,0,0,0,0,0,0,0
25879,16495,3,2016,42,28,0.039437,10,11,0,0,...,0,0,1,0,0,0,0,0,0,0
36606,12500,7,2011,136,103,0.021906,10,1,0,0,...,0,0,0,0,0,0,0,0,0,0
83,37900,7,2016,23,52,0.020966,9,10,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70103,8450,6,2013,121,44,0.047978,11,1,0,1,...,0,0,0,0,0,0,0,0,0,0
28826,2350,3,2007,101,1,0.107447,8,8,0,0,...,0,0,0,0,0,0,1,0,0,0
33077,9450,3,2016,0,2,0.000000,10,10,0,0,...,0,0,1,0,0,0,0,0,0,0
702,8900,6,2011,158,57,0.054785,11,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X_test

,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month,BMW,Citroen,...,Opel,Peugeot,Renault,SEAT,Skoda,Suzuki,Toyota,Volkswagen,Volvo,n
8327,17400,2,2013,18,2,0.070661,11,11,0,0,...,0,0,0,0,0,0,0,0,0,0
20492,1250,8,2000,36,3,0.078261,10,10,0,0,...,0,1,0,0,0,0,0,0,0,0
19904,3495,1,2009,39,26,0.020679,7,8,0,0,...,0,0,0,0,0,0,0,0,0,0
72152,17445,1,2016,58,27,0.050130,9,10,0,0,...,0,0,0,0,0,0,0,1,0,0
14374,14250,6,2015,30,42,0.036855,7,9,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73122,12400,4,2017,0,1,0.000000,9,9,0,0,...,0,0,0,0,1,0,0,0,0,0
49563,4450,7,2008,6,44,0.017442,11,12,0,0,...,0,1,0,0,0,0,0,0,0,0
34423,18950,5,2012,4,6,0.015810,8,8,0,0,...,0,0,0,0,0,0,0,0,0,0
12551,14745,1,2012,49,60,0.062905,9,11,0,0,...,0,0,0,0,0,0,1,0,0,0


In [20]:
#getting copies of the data to be scaled 
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

In [21]:
#feature scaling 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#features to be normalized
sc = ['price', 'first_registration_year', 'detail_views',
       'stock_days', 'ctr']

X_train_scaled[sc] = scaler.fit_transform(X_train[sc])
X_test_scaled[sc] = scaler.transform(X_test[sc])

In [22]:
X_train_scaled

,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month,BMW,Citroen,...,Opel,Peugeot,Renault,SEAT,Skoda,Suzuki,Toyota,Volkswagen,Volvo,n
63518,-0.127806,3,-0.321191,1.261898,2.507822,-0.014949,7,11,0,0,...,0,0,0,0,0,0,0,0,0,1
49793,0.600693,5,-2.934026,0.236722,-0.652280,-0.032094,11,11,1,0,...,0,0,0,0,0,0,0,0,0,0
25879,0.085688,3,0.754682,-0.223131,-0.249522,-0.033133,10,11,0,0,...,0,0,1,0,0,0,0,0,0,0
36606,-0.157652,7,-0.013799,0.173440,2.074083,-0.055355,10,1,0,0,...,0,0,0,0,0,0,0,0,0,0
83,1.389494,7,0.754682,-0.303288,0.494031,-0.056546,9,10,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70103,-0.404343,6,0.293593,0.110157,0.246180,-0.022305,11,1,0,1,...,0,0,0,0,0,0,0,0,0,0
28826,-0.775902,3,-0.628584,0.025781,-1.086020,0.053080,8,8,0,0,...,0,0,0,0,0,0,1,0,0,0
33077,-0.343432,3,0.754682,-0.400321,-1.055038,-0.083124,10,10,0,0,...,0,0,1,0,0,0,0,0,0,0
702,-0.376933,6,-0.013799,0.266254,0.648938,-0.013676,11,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
X_test_scaled

,price,first_zip_digit,first_registration_year,detail_views,stock_days,ctr,created_month,deleted_month,BMW,Citroen,...,Opel,Peugeot,Renault,SEAT,Skoda,Suzuki,Toyota,Volkswagen,Volvo,n
8327,0.140813,2,0.293593,-0.324383,-1.055038,0.006449,11,11,0,0,...,0,0,0,0,0,0,0,0,0,0
20492,-0.842904,8,-1.704457,-0.248444,-1.024057,0.016083,10,10,0,0,...,0,1,0,0,0,0,0,0,0,0
19904,-0.706158,1,-0.321191,-0.235787,-0.311485,-0.056911,7,8,0,0,...,0,0,0,0,0,0,0,0,0,0
72152,0.143554,1,0.754682,-0.155629,-0.280503,-0.019578,9,10,0,0,...,0,0,0,0,0,0,0,1,0,0
14374,-0.051058,6,0.600986,-0.273757,0.184218,-0.036405,7,9,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73122,-0.163744,4,0.908378,-0.400321,-1.086020,-0.083124,9,9,0,0,...,0,0,0,0,1,0,0,0,0,0
49563,-0.647988,7,-0.474888,-0.375008,0.246180,-0.061014,11,12,0,0,...,0,1,0,0,0,0,0,0,0,0
34423,0.235225,5,0.139897,-0.383446,-0.931113,-0.063082,8,8,0,0,...,0,0,0,0,0,0,0,0,0,0
12551,-0.020907,1,0.139897,-0.193599,0.741883,-0.003383,9,11,0,0,...,0,0,0,0,0,0,1,0,0,0
